In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
# Add the project root (toy-wm directory) to sys.path so we can import src as a package
# This allows relative imports within the package to work correctly
# Assumes notebook is run from the project root directory
project_root = os.getcwd()  # or use: os.path.dirname(os.path.abspath(''))
print(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch as t

# Import using absolute path from src package
from src.utils.checkpoint import load_model_from_config

#model = load_model_from_config("configs/small_6frame.yaml")
model = load_model_from_config("configs/poc_30frame_causal.yaml")
model.eval()

In [ ]:
from src.datasets.pong1m import get_loader

loader, pred2frame = get_loader(duration=1, fps=30, mode='-1,1')
frames, actions = next(iter(loader))
frames = pred2frame(frames)

In [ ]:
from src.trainers.diffusion_forcing import sample
import inspect
inspect.getsource(sample)
import matplotlib.pyplot as plt

z = sample(model, t.randn((100, 1, 3, 24, 24), device=model.device), actions=t.zeros(100, 1, dtype=t.int32, device=model.device), num_steps=4)
print(z.min(),z.max())
z = pred2frame(z)
print(z.min(),z.max())

num_images = 100
img_h, img_w = z.shape[-2], z.shape[-1]
grid_size = 10

fig, axes = plt.subplots(grid_size, grid_size, figsize=(12,12))
for idx in range(num_images):
    row = idx // grid_size
    col = idx % grid_size
    img = z[idx, 0].permute(1, 2, 0).cpu().numpy()  # shape: (24, 24, 3)
    axes[row, col].imshow(img)
    axes[row, col].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
def render_video(actions, start_frame = None, n_steps=8, cfg=0, negative_actions=None, clamp=True):
    frames = t.randn(1, 30, 3, 24, 24, device=model.device)
    if start_frame is not None:
        frames[:, 0] = start_frame
        start_idx = 1
    else:
        start_idx = 0
    for aidx in range(start_idx, actions.shape[1]):
        z = sample(model, frames, actions, num_steps=n_steps, cfg=cfg, negative_actions=negative_actions)
        frames[:, aidx:aidx+1] = z[:, aidx:aidx+1]
        if clamp:
            frames[:, aidx:aidx+1] = frames[:, aidx:aidx+1].clamp(-1, 1)
    return frames

In [ ]:
model = t.compile(model)

In [ ]:
actions = t.tensor([[0]+29*[2]], dtype=t.int32, device=model.device)
negative_actions = t.tensor([[0]+29*[3]], dtype=t.int32, device=model.device)
z = render_video(actions, n_steps=25, cfg=2, negative_actions=negative_actions, clamp=False)
frames = pred2frame(z)

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# z[0] shape: (num_frames, height, width, 3), where num_frames = frames.shape[1]
frames_np = frames[0].permute(0, 2, 3, 1).cpu().numpy()  # shape: (num_frames, height, width, 3)

fig, ax = plt.subplots()
im = ax.imshow(frames_np[0])
ax.set_title('Frame 0')

def animate(i):
    im.set_data(frames_np[i])
    ax.set_title(f'Frame {i}')
    return [im]

ani = animation.FuncAnimation(
    fig, animate, frames=frames_np.shape[0],
    interval=200, blit=True, repeat=True
)

plt.show()

# Optional: To save the gif uncomment below (requires pillow, install with `pip install pillow`)
from collections import Counter
# Majority vote over values in actions for the filename
if actions is not None:
    # Flatten actions tensor and convert to list
    action_vals = actions.flatten().tolist()
    majority_val = Counter(action_vals).most_common(1)[0][0]
    ani.save(f"sample_{majority_val}.gif", writer="pillow")
else:
    ani.save("sample_unknown.gif", writer="pillow")




In [ ]:
actions = t.tensor([6*[2]], dtype=t.int32, device="mps")
frames = render_video(actions, z[:, 0])
frames = pred2frame(frames)

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# z[0] shape: (num_frames, height, width, 3), where num_frames = frames.shape[1]
frames_np = frames[0].permute(0, 2, 3, 1).cpu().numpy()  # shape: (num_frames, height, width, 3)

fig, ax = plt.subplots()
im = ax.imshow(frames_np[0])
ax.set_title('Frame 0')

def animate(i):
    im.set_data(frames_np[i])
    ax.set_title(f'Frame {i}')
    return [im]

ani = animation.FuncAnimation(
    fig, animate, frames=frames_np.shape[0],
    interval=200, blit=True, repeat=True
)

plt.show()

# Optional: To save the gif uncomment below (requires pillow, install with `pip install pillow`)
ani.save("sample_cont.gif", writer="pillow")